In [1]:
import pandas as pd
import os
import json
import torch
import numpy as np
from torch.utils.data import DataLoader
from steams.data import KVyQVx
from steams.models.attention import dpnn4
from steams.trainer import AttentionTrainer,train, evaluation, prediction_prime, ensemble_prime
from steams.criterion import R2,RMSE,variance
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import geopandas as gpd

In [2]:
# parent_path = os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir))
# parent_dir = os.path.join(parent_path,'results','NWnn3')
# if not os.path.exists(parent_dir):
#     os.makedirs(parent_dir)

parent_dir = os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir))


In [3]:
resdir = os.path.join(parent_dir,'results')
figdir = os.path.join(parent_dir,'fig')
if not os.path.exists(figdir):
    os.makedirs(figdir)
if not os.path.exists(resdir):
    os.makedirs(resdir)

In [6]:
if torch.cuda.is_available():
    cuda_name = "1"
    num_workers=0
    pin_memory = True
    device = torch.device('cuda'+":"+cuda_name)
    #device = torch.device('cuda')
else:
    num_workers = 4
    pin_memory = False
    device = torch.device('cpu')

In [7]:
device

device(type='cpu')

## train/valid dataset

In [8]:
params = {
        "class" : "KVyQVx",
        "Y":{
          "path": "",
          "KEY":["x","y"],
          "VALUE": ["sensor_hq"],
          "nb_location" : 1000,
          "history_length": 0,
          "nb_sampling": 100
        },
        "X":{
          "path": "",
          "QUERY":["x","y"],
          "VALUE": ["sensor_hq"],
          "nb_location" : 1000,
          "gap_length": 0,
          "horizon_length":0,
          "nb_sampling": 100
        }
    }

In [9]:
params['Y']['path'] = os.path.join(parent_dir,"session","synth_1000_Y_train")
params['X']['path'] = os.path.join(parent_dir,"session","synth_1000_X_train")

train_dataset = KVyQVx(params)

In [10]:
params['Y']['path'] = os.path.join(parent_dir,"session","synth_1000_Y_valid")
params['X']['path'] = os.path.join(parent_dir,"session","synth_1000_X_valid")

valid_dataset = KVyQVx(params)

In [11]:
#train_dataset[0]

## model, optimizer, criterion, scheduler

In [12]:
#model
model = dpnn4(input_k=2,input_q=2, input_v=1, hidden_size=32)

In [15]:
# optimzer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) #8e-3
#optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# criterion
criterion = torch.nn.MSELoss()

# AttentionTrainer object
obj = AttentionTrainer(model,device)
obj.init_optimizer(optimizer)
obj.init_criterion(criterion)

## Training

In [16]:
# training
train(obj,train_dataset,valid_dataset,niter=200,n_iter_stop=10,batch_size=64,shuffle=True,num_workers=num_workers,pin_memory=pin_memory,resdir=None)

  epoch     train     valid
0     0  0.228426  0.109649
  epoch     train     valid
0     1  0.220762  0.106967
  epoch     train     valid
0     2  0.221312  0.110791
  epoch     train     valid
0     3  0.223956  0.109294
  epoch     train    valid
0     4  0.223679  0.10902
  epoch    train     valid
0     5  0.22123  0.110861
  epoch     train     valid
0     6  0.224799  0.111207
  epoch     train     valid
0     7  0.223122  0.108888
  epoch     train     valid
0     8  0.221142  0.110108
  epoch     train     valid
0     9  0.223942  0.111031
  epoch     train     valid
0    10  0.219862  0.112981
  epoch     train     valid
0    11  0.226266  0.112596
--> early stopping


In [17]:
obj.save_model(resdir,"1d-dpnn4")

## Or load model

In [18]:
obj.load_model(resdir,"1d-dpnn4")

/home/jean-marie/dev/01-packages/steams_pkg/steams/trainer.py:265: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path))


## Evaluation

In [19]:
params['Y']['path'] = os.path.join(parent_dir,"session","synth_1000_Y_eval")
params['X']['path'] = os.path.join(parent_dir,"session","synth_1000_X_eval")

eval_dataset = KVyQVx(params)

In [20]:
criterion = R2()
obj.init_criterion(criterion)

In [21]:
evaluation(obj,eval_dataset,batch_size=64,shuffle=True,num_workers=num_workers,pin_memory=pin_memory,resdir=None)

0.831004161387682


In [22]:
criterion = RMSE()
obj.init_criterion(criterion)

In [23]:
evaluation(obj,eval_dataset,batch_size=64,shuffle=True,num_workers=num_workers,pin_memory=pin_memory,resdir=None)

40.96903204917908


In [23]:
criterion = variance()
obj.init_criterion(criterion)

In [24]:
evaluation(obj,eval_dataset,batch_size=64,shuffle=True,num_workers=num_workers,pin_memory=pin_memory,resdir=None)

40.70465588569641


## QQplot

In [25]:
params['Y']['path'] = os.path.join(parent_dir,"session","synth_1000_Y_eval")
params['X']['path'] = os.path.join(parent_dir,"session","synth_1000_X_eval")

eval_dataset = KVyQVx(params)

In [26]:
results = prediction_prime(obj,eval_dataset)

/home/jean-marie/dev/01-packages/steams_pkg/steams/trainer.py:163: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results,tmp_df],ignore_index=True,axis=0)


In [27]:
pred_name = ['pred_' + v for v in eval_dataset.VALUE_X]

fig, ax= plt.subplots(figsize=(14, 12))
ax.scatter(results.loc[:,eval_dataset.VALUE_X],results.loc[:,pred_name], linewidth=2.0,c="black")
ax.axline((0, 0), slope=1., color='blue')
# ax.set_xlim(0, 2.5)
ax.set_ylim(results.loc[:,pred_name].min().item()-0.5, results.loc[:,pred_name].max().item()+0.5)
ax.set(xlabel='observations', ylabel='predictions')

fig_filename = os.path.join(parent_dir,'fig','1c-qqplot.png')
plt.savefig(fig_filename, dpi = 300)

## Illustration of a 2d-map prediction

In [38]:
params = {
        "class" : "KVyQVx",
        "Y":{
          "path": "",
          "KEY":["x","y"],
          "VALUE": ["sensor_hq"],
          "nb_location" : 1000,
          "history_length": 0,
          "nb_sampling": 100
        },
        "X":{
          "path": "",
          "QUERY":["x","y"],
          "VALUE": ["ref"],
          "nb_location" : 6400,
          "gap_length": 0,
          "horizon_length":0,
          "nb_sampling": 6400
        }
    }

In [39]:
params['Y']['path'] = os.path.join(parent_dir,"session","synth_1000_Y_eval")
params['X']['path'] = os.path.join(parent_dir,"session","synth_all_6400")

eval_dataset = KVyQVx(params)

In [40]:
eval_dataset.scale(True)
KEY_Y,VALUE_Y,QUERY_X, VALUE_X = eval_dataset[0]
QUERY_res, VALUE_res = obj.predict(KEY_Y,VALUE_Y,QUERY_X,eval_dataset)

In [41]:
# output
VALUE_X = torch.reshape(VALUE_X,(1,VALUE_X.shape[0],VALUE_X.shape[1]))
#QUERY_X_unscale = eval_dataset.unscale(QUERY_X,"QUERY").to(device)
VALUE_X_unscale = eval_dataset.unscale(VALUE_X,"VALUE_X").to(device)

In [42]:
tmp = np.concatenate((QUERY_res, VALUE_res,VALUE_X_unscale),2)
tmp = np.reshape(tmp,(tmp.shape[1],tmp.shape[2]))
results = pd.DataFrame(tmp,columns=['x','y','values','truth'])   

In [43]:
# predictors
VALUE_Y = torch.reshape(VALUE_Y,(1,VALUE_Y.shape[0],VALUE_Y.shape[1]))
KEY_Y = torch.reshape(KEY_Y,(1,KEY_Y.shape[0],KEY_Y.shape[1]))
KEY_Y_unscale = eval_dataset.unscale(KEY_Y,"KEY").to(device)
VALUE_Y_unscale = eval_dataset.unscale(VALUE_Y,"VALUE_Y").to(device)

tmp = np.concatenate((KEY_Y_unscale, VALUE_Y_unscale),2)
tmp = np.reshape(tmp,(tmp.shape[1],tmp.shape[2]))
predictors = pd.DataFrame(tmp,columns=['x','y','values'])        

In [44]:
vmax = np.array(
    results['values'].max(),
    results['truth'].max()).max()

vmin = np.array(
    results['values'].min(),
    results['truth'].min()).min()

In [45]:
foo = predictors.copy()

In [46]:
results

x          y      values       truth
0     4350787.5  4097312.5  203.078003  414.594116
1     4353587.5  4089812.5  198.897964   48.107864
2     4345462.5  4093237.5  199.702545  127.239334
3     4354562.5  4098162.5  200.866409  217.373062
4     4351962.5  4097162.5  203.078918  388.368896
...         ...        ...         ...         ...
6395  4350912.5  4096762.5  203.016068  414.827850
6396  4350087.5  4090837.5  200.122650  245.009338
6397  4354912.5  4098487.5  200.679886  231.907196
6398  4345912.5  4094412.5  202.028641  343.515503
6399  4345212.5  4092062.5  199.996613  169.932999

[6400 rows x 4 columns]

In [47]:
fig = plt.figure(constrained_layout=True,figsize=(12, 3))
gs = GridSpec(1, 2, figure=fig)

ax1 = fig.add_subplot(gs[0,0])
c1 = ax1.scatter(x=results["x"], y=results["y"], s=2,c=results['values'],vmin=vmin,vmax=vmax)
#ax1.set(ylabel='Y (meters)')
ax1.set(xlabel='X (meters)')
plt.gca().set_aspect("equal")

ax2 = fig.add_subplot(gs[0,1])
c2 = ax2.scatter(x=results["x"], y=results["y"], s=2,c=results['truth'],vmin=vmin,vmax=vmax)
#ax2.set(ylabel='Y (meters)')
ax2.set(xlabel='X (meters)')
plt.gca().set_aspect("equal")

fig.colorbar(c1, ax=ax1,label ='meters')
fig.colorbar(c2, ax=ax2,label ='meters')

fig_filename = os.path.join(parent_dir,'fig','illustration_prediction.png')
plt.savefig(fig_filename, dpi = 300)
